# Análise das Mudanças Climáticas no Rio Grande do Sul
Este notebook executa o pipeline completo para investigar mudanças sazonais, variabilidade e extremos de temperatura no Rio Grande do Sul (RS) usando os dados do **BDMEP/INMET**.

**Fluxo geral:**
1. Configuração do ambiente e parâmetros.
2. Leitura automática dos arquivos CSV das estações.
3. Pré‑processamento e agregação diária/mensal/sazonal.
4. Cálculo de indicadores (médias, desvios, extremos, anomalias).
5. Análise de tendências (regressão linear, Mann‑Kendall).
6. Visualizações (séries temporais, boxplots, mapas de anomalia).

> **Dica:** copie seus CSVs para a pasta `./dados` antes de executar.

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import scipy.stats as stats
from pathlib import Path
from datetime import datetime
import pymannkendall as mk
from tqdm import tqdm
import time
import io

# Garante que os gráficos apareçam no notebook
%matplotlib inline

In [14]:
# --- Configurações do usuário ---
DATA_DIR = Path('./dados')          # Pasta com os CSVs BDMEP
SHAPE_PATH = Path('./RS.shp')       # Shapefile do contorno do RS
OUT_DIR = Path('./outputs')         # Pasta de saída
BASELINE_START = 1961               # Início do período de referência
BASELINE_END   = 1990               # Fim do período de referência
OUT_DIR.mkdir(exist_ok=True)
print(f'Pasta de dados: {DATA_DIR.resolve()}')
print(f'Pasta de saídas: {OUT_DIR.resolve()}')

Pasta de dados: /home/vitorklock/visualStudio/projects/study/unijui/PI/2025/clima-rs/dados
Pasta de saídas: /home/vitorklock/visualStudio/projects/study/unijui/PI/2025/clima-rs/outputs


In [15]:
def load_bdmepr_csv(path: Path) -> pd.DataFrame:
    """
    Carrega um CSV do BDMEP (versão 5 - Corrigida com os cabeçalhos do usuário).
    Usa a dica da "linha dupla" para separar metadados e dados, e usa os nomes
    corretos de colunas fornecidos pelo usuário.
    """
    # --- Passo 1: Ler o arquivo e separar metadados e dados ---
    try:
        raw_content = path.read_text(encoding='latin1', errors='ignore')
    except UnicodeDecodeError:
        raw_content = path.read_text(encoding='utf-8', errors='ignore')

    parts = raw_content.replace('\r\n', '\n').split('\n\n', 1)
    if len(parts) != 2:
        raise ValueError(f"Não foi possível encontrar o separador de linha dupla no arquivo: {path.name}")
    metadata_block, data_block = parts
    
    # --- Passo 2: Extrair metadados ---
    metadata = {}
    for line in metadata_block.split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            metadata[key.strip().upper()] = value.strip()

    # --- Passo 3: Ler os dados tabulares CORRETAMENTE ---
    df = pd.read_csv(
        io.StringIO(data_block),
        sep=';',
        decimal=',',
        # Trata 'null', '-9999', etc. como valores Nulos/NaN
        na_values=['null', '-9999', '9999.9'],
        low_memory=False
    )
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # --- Passo 4: Renomear colunas com o mapeamento CORRETO ---
    rename_map = {
        # ⇢ grafias com vírgula:
        'Data Medicao': 'date_str',
        'PRECIPITACAO TOTAL, DIARIO (AUT)(mm)': 'precip',
        'TEMPERATURA MEDIA, DIARIA (AUT)(°C)': 'tmedia',
        'TEMPERATURA MAXIMA, DIARIA (AUT)(°C)': 'tmax',
        'TEMPERATURA MINIMA, DIARIA (AUT)(°C)': 'tmin',

        # ⇢ grafias SEM vírgula:
        'TEMPERATURA MEDIA DIARIA (AUT)(°C)': 'tmedia',
        'TEMPERATURA MAXIMA DIARIA (AUT)(°C)': 'tmax',
        'TEMPERATURA MINIMA DIARIA (AUT)(°C)': 'tmin',
        'PRECIPITACAO TOTAL DIARIA (AUT)(mm)': 'precip',

        # ⇢ outras variações comuns encontradas em séries antigas:
        'TEMPERATURA MEDIA DIARIA (°C)': 'tmedia',
        'TEMPERATURA MAXIMA DIARIA (°C)': 'tmax',
        'TEMPERATURA MINIMA DIARIA (°C)': 'tmin',
        'TEMPERATURA MEDIA COMPENSADA (°C)': 'tmedia',
        'PRECIPITACAO TOTAL, DIARIA (mm)': 'precip',
        'DATA': 'date_str', 'Data': 'date_str',
    }
    # Renomeia as colunas encontradas no dicionário
    df = df.rename(columns=lambda c: rename_map.get(c.strip(), c.strip()))
    
    # --- Passo 5: Processar data e variáveis ---
    # Se a coluna de data não foi renomeada, assume que é a primeira
    if 'date_str' not in df.columns and len(df.columns) > 0:
        df = df.rename(columns={df.columns[0]: 'date_str'})

    df['date'] = pd.to_datetime(df['date_str'], errors='coerce')
    if df['date'].isna().all(): raise ValueError(f"Não foi possível converter as datas para DateTime no arquivo {path.name}")
    df = df.set_index('date').sort_index()

    # LÓGICA CHAVE: Calcular tmedia se não existir
    if 'tmedia' not in df.columns and 'tmax' in df.columns and 'tmin' in df.columns:
        df['tmedia'] = (df['tmax'] + df['tmin']) / 2
        
    # Garante que as colunas são numéricas e aplica limites físicos
    final_cols = ['tmedia', 'tmax', 'tmin', 'precip']
    for col in final_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            if col.startswith('t'):
                df[col] = df[col].where((df[col] > -50) & (df[col] < 50))
            elif col == 'precip':
                df[col] = df[col].where(df[col] >= 0)

    # --- Passo 6: Guardar metadados e retornar ---
    df.attrs['station_code'] = metadata.get('CODIGO OMM', metadata.get('CODIGO', path.stem.split('_')[1]))
    df.attrs['latitude'] = pd.to_numeric(metadata.get('LATITUDE'), errors='coerce')
    df.attrs['longitude'] = pd.to_numeric(metadata.get('LONGITUDE'), errors='coerce')
    df.attrs['filename'] = path.name
    
    return df

# --- Bloco de Execução Principal ---

In [16]:
all_files = list(DATA_DIR.glob('*.csv'))
if not all_files:
    raise FileNotFoundError(f"Nenhum arquivo CSV encontrado em {DATA_DIR}")

stations_meta = []        # onde vamos armazenar as coordenadas etc.
data_dict      = {}       # df diários completos por estação
failed_files   = []

for f in tqdm(all_files, desc='Lendo estações'):
    try:
        df_station = load_bdmepr_csv(f)
    
        code = df_station.attrs['station_code']
        data_dict[code] = df_station

        stations_meta.append({
            'code'     : code,
            'name'     : df_station.attrs.get('station_name', f.stem.split('_')[0]),
            'lat'      : df_station.attrs['latitude'],
            'lon'      : df_station.attrs['longitude'],
            'alt'      : df_station.attrs.get('altitude', np.nan),
            'data_ini' : df_station.index.min(),
            'data_fim' : df_station.index.max(),
        })


        
    except Exception as e:
        failed_files.append((f.name, str(e)))
        continue

if failed_files:
    print("\nAVISO: Alguns arquivos não puderam ser carregados:")
    for fname, error in failed_files:
        print(f"- {fname}: {error}")

print(f'\n{len(data_dict)} estações carregadas com sucesso.')

Lendo estações: 100%|██████████| 43/43 [00:00<00:00, 91.83it/s]


43 estações carregadas com sucesso.


## 2. GeoDataFrame de metadados (para mapas)

In [17]:
meta_df = pd.DataFrame(stations_meta).dropna(subset=['lat', 'lon'])
gdf_meta = gpd.GeoDataFrame(
    meta_df,
    geometry=gpd.points_from_xy(meta_df.lon, meta_df.lat),
    crs="EPSG:4326"
)
print(gdf_meta.head())

   code   name        lat        lon  alt   data_ini   data_fim  \
0  A854  dados -27.395556 -53.429444  NaN 2007-12-13 2025-01-01   
1  A844  dados -28.222381 -51.512845  NaN 2007-03-01 2025-01-01   
2  A833  dados -29.191599 -54.885653  NaN 2009-02-03 2025-01-01   
3  A831  dados -30.368611 -56.437222  NaN 2007-10-16 2025-01-01   
4  A827  dados -31.347778 -54.013333  NaN 2007-01-03 2025-01-01   

                      geometry  
0  POINT (-53.42944 -27.39556)  
1  POINT (-51.51284 -28.22238)  
2   POINT (-54.88565 -29.1916)  
3  POINT (-56.43722 -30.36861)  
4  POINT (-54.01333 -31.34778)  


## 3. Funções auxiliares: estação do ano, climatologia-base e agregações

In [18]:
# %%
def add_southern_season(df):
    """Adiciona colunas de ano-mês-estação (hemisfério sul)."""
    d = df.copy()
    d['year']  = d.index.year
    d['month'] = d.index.month

    m2s = {12:'summer',1:'summer',2:'summer',
            3:'autumn',4:'autumn',5:'autumn',
            6:'winter',7:'winter',8:'winter',
            9:'spring',10:'spring',11:'spring'}
    d['season'] = d['month'].map(m2s)
    d.loc[d['month']==12, 'year'] += 1       # dez pertence ao verão do ano seguinte
    return d

def seasonal_summary(df):
    """
    Gera quadro de métricas sazonais **apenas** para variáveis presentes.
    - tmedia_mean / tmedia_sd
    - tmax_90p   : nº de dias acima do percentil 90 da própria série
    - tmin_10p   : nº de dias abaixo do percentil 10
    - precip_sum : soma da precipitação
    """
    d = add_southern_season(df)

    agg = {}
    if 'tmedia' in d.columns:
        agg['tmedia_mean'] = ('tmedia', 'mean')
        agg['tmedia_sd'  ] = ('tmedia', 'std')
    if 'tmax' in d.columns:
        agg['tmax_90p'] = ('tmax',  lambda s: (s > s.quantile(0.90)).sum())
    if 'tmin' in d.columns:
        agg['tmin_10p'] = ('tmin',  lambda s: (s < s.quantile(0.10)).sum())
    if 'precip' in d.columns:
        agg['precip_sum'] = ('precip','sum')

    return (
        d.groupby(['year','season'])
         .agg(**agg)
         .reset_index()
    )

def climatology_base(seasonal, start=1961, end=1990):
    """
    Climatologia 1961-1990 da temperatura média.
    Se 'tmedia_mean' não existir (ou não houver anos dentro do intervalo),
    devolve apenas a lista de estações com NaN — assim o merge não quebra.
    """
    if 'tmedia_mean' not in seasonal.columns:
        # devolve seasons únicas com NaN
        return (seasonal[['season']]
                .drop_duplicates()
                .assign(clim_tmedia=np.nan))

    base = seasonal.query("@start <= year <= @end").dropna(subset=['tmedia_mean'])
    if base.empty:
        return (seasonal[['season']]
                .drop_duplicates()
                .assign(clim_tmedia=np.nan))

    return (
        base.groupby('season')
            .agg(clim_tmedia=('tmedia_mean','mean'))
            .reset_index()
    )


## 4. Calcula agregados e anomalias para cada estação

In [19]:
# %% Calculo das métricas sazonais + anomalias
seasonal_dict = {}
anomaly_dict  = {}

for code, df in tqdm(data_dict.items(), desc='Calculando sazonais'):
    seas = seasonal_summary(df)                       # métricas da estação
    clim = climatology_base(seas, BASELINE_START, BASELINE_END)

    # junta climatologia (pode vir SEM 'clim_tmedia' se não houver tmedia)
    seas = seas.merge(clim, on='season', how='left')

    # calcula anomalia só se ambas existirem
    if {'tmedia_mean', 'clim_tmedia'}.issubset(seas.columns):
        seas['tmedia_anom'] = seas['tmedia_mean'] - seas['clim_tmedia']
    else:
        seas['tmedia_anom'] = np.nan                  # placeholder

    seasonal_dict[code] = seas
    anomaly_dict[code]  = seas[['year', 'season', 'tmedia_anom']]


Calculando sazonais: 100%|██████████| 43/43 [00:00<00:00, 123.65it/s]


In [20]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

plt.rcParams.update({"figure.dpi": 110})

In [21]:
def climatology_base(seasonal: pd.DataFrame, start: int | None = 1961,
                      end: int | None = 1990, *, fallback_years: int = 5) -> pd.DataFrame:
    """Retorna a climatologia (média  da *tmedia_mean*) por estação do ano.

    - Se houver pelo menos 5 anos dentro do intervalo *start‑end* ➜ usa‑se esse
      período oficial.
    - Caso contrário, calcula‑se a média dos **primeiros `fallback_years` anos
      disponíveis** (mínimo de 5 anos também).  Se ainda assim não houver dados
      suficientes, devolve‐se *NaN*.
    """
    if 'tmedia_mean' not in seasonal.columns:
        return (seasonal[['season']].drop_duplicates().assign(clim_tmedia=pd.NA))

    # ── 1. Tentativa com período oficial ──────────────────────────────────────
    if start is not None and end is not None:
        official = seasonal.query("@start <= year <= @end and tmedia_mean.notna()")
        if official['year'].nunique() >= 5:
            return (official.groupby('season')
                           .agg(clim_tmedia=('tmedia_mean', 'mean'))
                           .reset_index())

    # ── 2. Fallback: primeiros N anos disponíveis ────────────────────────────
    min_year = seasonal['year'].min()
    max_fallback = min_year + fallback_years - 1
    fallback_df = seasonal.query("@min_year <= year <= @max_fallback and tmedia_mean.notna()")
    if fallback_df['year'].nunique() >= 5:
        return (fallback_df.groupby('season')
                          .agg(clim_tmedia=('tmedia_mean', 'mean'))
                          .reset_index())

    # ── 3. Sem dados suficientes ─────────────────────────────────────────────
    return (seasonal[['season']].drop_duplicates().assign(clim_tmedia=pd.NA))



def plot_rs_mean_anomaly(seasonal_dict: dict[str, pd.DataFrame], *, save: bool = False):
    """Plota a anomalia média estadual por estação do ano ao longo dos anos."""
    frames = [df[['year', 'season', 'tmedia_anom']].dropna() for df in seasonal_dict.values()
              if 'tmedia_anom' in df.columns]

    if not frames:
        print("⚠️  Nenhuma anomalia calculada. Verifique as séries ou o cálculo da climatologia.")
        return

    all_anom = pd.concat(frames, ignore_index=True)
    if all_anom.empty:
        print("⚠️  DataFrame de anomalias ficou vazio após concatenação.")
        return

    mean_anom = (all_anom.groupby(['year', 'season'])['tmedia_anom']
                        .mean().unstack().sort_index())

    fig, ax = plt.subplots(figsize=(10, 5))
    mean_anom.plot(ax=ax, marker="o")
    ax.axhline(0, color="gray", lw=0.8)
    ax.set_ylabel("Anomalia de T média (°C)")
    ax.set_xlabel("Ano")
    ax.set_title("Anomalia média estadual da temperatura por estação do ano")
    ax.legend(title="Estação")
    ax.grid(True, ls=":", lw=0.5)

    if save:
        fig.savefig(OUT_DIR / "serie_anomalia_media_RS.png", bbox_inches="tight")
    plt.show()


In [22]:
plot_rs_mean_anomaly(seasonal_dict, save=True)

⚠️  DataFrame de anomalias ficou vazio após concatenação.


In [23]:
seasonal_dict = {}
for code, df in data_dict.items():
    seas = seasonal_summary(df)
    clim = climatology_base(seas, BASELINE_START, BASELINE_END, fallback_years=10)
    seas = seas.merge(clim, on='season', how='left')
    seas['tmedia_anom'] = seas['tmedia_mean'] - seas['clim_tmedia']
    seasonal_dict[code] = seas

anomaly_dict = {c: d[['year','season','tmedia_anom']] for c,d in seasonal_dict.items()}

KeyError: 'tmedia_mean'

In [ ]:
map_recent_mean_anomaly(seasonal_dict, gdf_meta, SHAPE_PATH, start_year=2015, save=True)